In [1]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

driver = webdriver.Firefox()
driver.get(premLeague)
matches = driver.find_elements(By.CSS_SELECTOR, ".sp-o-market__title > a")
matches_links = []

# Get links for the matches
# [[match, link]]
for match in matches:
    matches_links.append([match.text, match.get_attribute('href')])

driver.close()

# For each link, we fetch the player data
# create an object [name, odds, match]
# details = [   name, match,
#              {"first":[first[0],first[1]]} ,
# {"anytime":[anytime[0],anytime[1]]},
# {"last":[last[0],last[1]]},
# {"two_or_more":[two_or_more[0],two_or_more[1]]},
# {"hattrick":[hattrick[0],hattrick[1]]},
# {"first_and_last":[first_and_last[0],first_and_last[1]]},
# {"first_or_last":[first_or_last[0],first_or_last[1]]}
# ]
details = []
for link in matches_links:  # Here you're getting data for each link .i.e each loop has one match
    driver = webdriver.Firefox()
    driver.get(link[1])
    players = driver.find_elements(
        By.CSS_SELECTOR, "article.cpt-goalscorer__player")

    if (len(players) == 0):  # check if the details are loaded. Otherwise, it is in the following GW
        continue

    players_details = []

    for index, tag in enumerate(players):  # for each match get player details
        # Get player name
        player = tag.find_element(
            By.TAG_NAME, 'h3').get_attribute("textContent")
        odds = tag.find_elements(By.CLASS_NAME, 'selectiondetails')

        def splitOdds(odd):
            if (len(odd.split('/')) > 1):
                return odd.split('/')
            else:
                return [odd]

        # Get players odds
        if (index+1 == len(players)):
            # For the no goal scored odd
            first = odds[0].get_attribute("data-odds")
            odss_details = {"name": player, "match": link[0], "first": first}

            players_details.append(odss_details)
            continue
        else:
            first = odds[0].get_attribute("data-odds")
            anytime = odds[1].get_attribute("data-odds")
            last = odds[2].get_attribute("data-odds")
            two_or_more = odds[3].get_attribute("data-odds")
            hattrick = odds[4].get_attribute("data-odds")
            first_and_last = odds[5].get_attribute("data-odds")
            first_or_last = odds[6].get_attribute("data-odds")

            odss_details = {
                "name": player,
                "match": link[0],
                "first": first,
                "anytime": anytime,
                "last": last,
                "two_or_more": two_or_more,
                "hattrick": hattrick,
                "first_and_last": first_and_last,
                "first_or_last": first_or_last
            }
            players_details.append(odss_details)

    # group the match details
    details.append(players_details)

    driver.close()


print(details)
print(matches_links)

[[{'name': 'Erling Haaland', 'match': 'Man City v Liverpool', 'first': '14/5', 'anytime': '7/10', 'last': '14/5', 'two_or_more': '18/5', 'hattrick': '14/1', 'first_and_last': '17/2', 'first_or_last': '29/20'}, {'name': 'Julian Alvarez', 'match': 'Man City v Liverpool', 'first': '6/1', 'anytime': '15/8', 'last': '6/1', 'two_or_more': '11/1', 'hattrick': '50/1', 'first_and_last': '20/1', 'first_or_last': '10/3'}, {'name': 'Jeremy Doku', 'match': 'Man City v Liverpool', 'first': '9/1', 'anytime': '16/5', 'last': '17/2', 'two_or_more': '20/1', 'hattrick': '125/1', 'first_and_last': '33/1', 'first_or_last': '11/2'}, {'name': 'Phil Foden', 'match': 'Man City v Liverpool', 'first': '9/1', 'anytime': '11/4', 'last': '8/1', 'two_or_more': '18/1', 'hattrick': '100/1', 'first_and_last': '28/1', 'first_or_last': '24/5'}, {'name': 'Oscar Bobb', 'match': 'Man City v Liverpool', 'first': '10/1', 'anytime': '16/5', 'last': '17/2', 'two_or_more': '20/1', 'hattrick': '125/1', 'first_and_last': '33/1', '

In [5]:
import pandas as pd
# len(det)
init_data = {"name": [],
             "match": [],
             "first": [],
             "anytime": [],
             "last": [],
             "two_or_more": [],
             "hattrick": [],
             "first_and_last": [],
             "first_or_last": []}

df = pd.DataFrame(init_data)

for items in details:  # for each match in object
    for item in items:  # for each player in match
        df.loc[len(df)] = item
with pd.ExcelWriter("../data/scrapped/williamhill.xlsx", mode="a") as writer:
    df.to_excel(writer, sheet_name="GW13")
# df.to_excel("../data/scrapped/williamhill_v1.xlsx", sheet_name="GW12")